In [10]:
import copy

import alns
import numpy as np
from functions.part1 import part1
from classes.deliveryrouting import DeliveryRouting

In [11]:
instance_dir = "data/0o50t75s1p100"
dr, final_result = part1(instance_dir)
dr.final_result = final_result

In [12]:
def random_destroy(State : DeliveryRouting, rnd_state : np.random.seed = np.random.seed(42)):
    destroyed = copy.deepcopy(State)

    order_destroyed = np.random.choice(destroyed.orders)
    destroyed.unassigned = order_destroyed


    for time, routes in destroyed.final_result.items():
        for route in routes:
            for bundle in route:
                if order_destroyed in bundle.bundle:
                    bundle.bundle.remove(order_destroyed)

    return destroyed

In [13]:
d = random_destroy(dr)